In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-auth google-auth-oauthlib google-api-python-client PyYAML

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-api-python-client google-auth google-auth-oauthlib requests urllib3 httplib2


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade google-auth-httplib2


In [ ]:
from __future__ import annotations
from pathlib import Path
import os, time, socket

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import httplib2
from google_auth_httplib2 import AuthorizedHttp   # ✅ authorize httplib2 with creds

SCOPES = ["https://www.googleapis.com/auth/tagmanager.readonly"]
CRED_PATH = Path(os.environ.get(
    "GTM_OAUTH_PATH",
    r"[YOUR_PATH]\client_secret.json"
))
TOKEN_PATH = Path(os.environ.get("GTM_TOKEN_PATH", "token.json"))

def get_gtm_service_oauth():
    if not CRED_PATH.exists():
        raise FileNotFoundError(f"OAuth client file not found: {CRED_PATH}")
    creds = None
    if TOKEN_PATH.exists():
        creds = Credentials.from_authorized_user_file(str(TOKEN_PATH), SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(str(CRED_PATH), SCOPES)
            creds = flow.run_local_server(port=0)
        TOKEN_PATH.write_text(creds.to_json(), encoding="utf-8")

    # ✅ Build a timeout-enabled HTTP and wrap it with your credentials
    raw_http = httplib2.Http(timeout=60)               # 60s timeout
    authed_http = AuthorizedHttp(creds, http=raw_http) # authorize transport

    # ✅ Pass ONLY http=..., NOT credentials=... (they are mutually exclusive)
    return build("tagmanager", "v2", http=authed_http, cache_discovery=False)

service = get_gtm_service_oauth()
print("✅ GTM client ready (OAuth)")



In [ ]:

# ---- 2) Robust execute with retries (use for all API calls) ----
TRANSIENT = (ConnectionAbortedError, ConnectionError, TimeoutError, socket.timeout)
def exec_with_retry(req, tries=5, backoff=1.5):
    for i in range(tries):
        try:
            return req.execute(num_retries=3)  # client-level retries too
        except HttpError as e:
            status = getattr(e, "status_code", getattr(getattr(e, "resp", None), "status", 0))
            if 500 <= (status or 0) < 600 and i < tries - 1:
                time.sleep(backoff ** i); continue
            raise
        except TRANSIENT:
            if i < tries - 1:
                time.sleep(backoff ** i); continue
            raise

# ---- 3) List helpers ----
def safe_filename(s: str) -> str:
    """Make a string safe for filenames."""
    return (
        s.replace(" ", "_")
         .replace("/", "_")
         .replace("\\", "_")
         .replace(":", "_")
    )
    
def list_accounts(service):
    return exec_with_retry(service.accounts().list()).get("account", [])

def list_containers(service, account_path):
    return exec_with_retry(service.accounts().containers().list(parent=account_path)).get("container", [])

def list_workspaces(service, container_path):
    return exec_with_retry(service.accounts().containers().workspaces().list(parent=container_path)).get("workspace", [])

def fetch_workspace_entities(service, workspace_path):
    tags = exec_with_retry(service.accounts().containers().workspaces().tags().list(parent=workspace_path)).get("tag", [])
    triggers = exec_with_retry(service.accounts().containers().workspaces().triggers().list(parent=workspace_path)).get("trigger", [])
    variables = exec_with_retry(service.accounts().containers().workspaces().variables().list(parent=workspace_path)).get("variable", [])
    builtins = exec_with_retry(service.accounts().containers().workspaces().built_in_variables().list(parent=workspace_path)).get("builtInVariable", [])
    return tags, triggers, variables, builtins

def fetch_live_version_entities(service, container_path):
    live = exec_with_retry(service.accounts().containers().versions().live(parent=container_path))
    cv = live.get("containerVersion") or {}
    return (
        cv.get("tag", []),
        cv.get("trigger", []),
        cv.get("variable", []),
        cv.get("builtInVariable", []),
        live.get("path"),
    )

# ---- 4) Matching + selection (reads env vars if args are None) ----
def _match(items, key_name, want: Optional[str]):
    if not items: return None
    if not want:  return items[0]
    want_l = str(want).lower()
    for it in items:
        if str(it.get(key_name, "")).lower() == want_l: return it
        if str(it.get("name", "")).lower() == want_l:  return it
    for it in items:  # partial
        if want_l in str(it.get("name","")).lower():   return it
    return None

def pick_gtm_objects(service, account=None, container=None, workspace=None):
    account = account or os.getenv("GTM_ACCOUNT")
    container = container or os.getenv("GTM_CONTAINER")
    workspace = workspace or os.getenv("GTM_WORKSPACE")

    accts = list_accounts(service)
    if not accts:
        raise RuntimeError("No GTM accounts visible to this user.")
    acct = _match(accts, "accountId", account)
    if not acct:
        raise RuntimeError(f"Account '{account}' not found. Available: {[a.get('name') for a in accts]}")
    account_path = acct["path"]

    conts = list_containers(service, account_path)
    if not conts:
        raise RuntimeError(f"No containers in account '{acct.get('name')}' ({acct.get('accountId')}).")
    cont = _match(conts, "containerId", container)
    if not cont:
        raise RuntimeError(f"Container '{container}' not found. Available: {[c.get('name') for c in conts]}")
    container_path = cont["path"]

    wss = list_workspaces(service, container_path)
    if not wss:
        raise RuntimeError(f"No workspaces in container '{cont.get('name')}'.")
    ws = _match(wss, "workspaceId", workspace) or next((w for w in wss if w.get("name")=="Default Workspace"), wss[0])
    workspace_path = ws["path"]

    print("✅ Selected:")
    print(f"  Account   : {acct.get('name')} (ID {acct.get('accountId')})  path={account_path}")
    print(f"  Container : {cont.get('name')} (ID {cont.get('containerId')}, usageContext={cont.get('usageContext')})  path={container_path}")
    print(f"  Workspace : {ws.get('name')} (ID {ws.get('workspaceId')})  path={workspace_path}")

    return acct, cont, ws, account_path, container_path, workspace_path

    tags, triggers, variables, builtins = fetch_workspace_entities(service, workspace_path)
    source = "workspace"

    if use_live_if_empty and len(tags) == 0 and len(triggers) == 0 and len(variables) == 0:
        tags, triggers, variables, builtins, live_path = fetch_live_version_entities(service, container_path)
        source = "live"
        print(f"ℹ️ Workspace empty, snapshotting LIVE instead: {live_path}")

    def norm_tag(t):
        status, ctypes = consent_info(t)
        need = required_consent_for_tag(t)
        return {
            "name": t.get("name",""),
            "type": t.get("type",""),
            "consent_status": status,
            "consent_types": sorted(ctypes),
            "consent_required": sorted(need),
        }

    def norm_trigger(tr):
        return {
            "name": tr.get("name",""),
            "type": tr.get("type",""),
        }

    def norm_variable(v):
        return {
            "name": v.get("name",""),
            "type": v.get("type",""),
        }

    snap = {
        "account": acct.get("name"),
        "container": cont.get("name"),
        "workspace": ws.get("name"),
        "source": source,   # "workspace" or "live"
        "tags": {t.get("name",""): norm_tag(t) for t in tags},
        "triggers": {tr.get("name",""): norm_trigger(tr) for tr in triggers},
        "variables": {v.get("name",""): norm_variable(v) for v in variables},
    }
    return snap


# ---- 5) Pick by env (or pass explicit names/IDs here) ----

os.environ["GTM_ACCOUNT"]   = os.environ.get("GTM_ACCOUNT", "YOUR ACCOUNT")         # or "12345678"
os.environ["GTM_WORKSPACE"] = os.environ.get("GTM_WORKSPACE", "Default Workspace")
os.environ["GTM_CONTAINER"] = os.environ.get("GTM_CONTAINER", "YOUR_CONTAINER_NAME")   # or "87654321"
os.environ["GTM_MARKET"] = os.environ.get("GTM_MARKET", "YOUR_MARKET")   

acct, cont, ws, account_path, container_path, workspace_path = pick_gtm_objects(service)

market = os.getenv("GTM_MARKET", "XX")

container_name = safe_filename(cont.get("name", "container"))
workspace_name = safe_filename(ws.get("name", "workspace"))

out_csv = f"{market}_{container_name}_{workspace_name}_gtm_audit_findings.csv"
out_yml = f"{market}_{container_name}_{workspace_name}_gtm_audit.report.yml"


# ---- 6) Fetch entities; fallback to LIVE if workspace empty ----
tags, triggers, variables, builtins = fetch_workspace_entities(service, workspace_path)
print(f"Workspace entities → tags={len(tags)}, triggers={len(triggers)}, variables={len(variables)}, builtIns={len(builtins)}")

container_export_like = {
    "containerVersion": {
        "tag": tags,
        "trigger": triggers,
        "variable": variables,
        "builtInVariable": builtins,
    }
}

if len(tags) == 0 and len(triggers) == 0 and len(variables) == 0:
    tags_live, triggers_live, variables_live, builtins_live, live_path = fetch_live_version_entities(service, container_path)
    print("ℹ️ Workspace looks empty; falling back to LIVE container version:")
    print(f"  live path: {live_path}")
    print(f"  LIVE entities → tags={len(tags_live)}, triggers={len(triggers_live)}, variables={len(variables_live)}, builtIns={len(builtins_live)}")
    container_export_like = {
        "containerVersion": {
            "tag": tags_live,
            "trigger": triggers_live,
            "variable": variables_live,
            "builtInVariable": builtins_live,
        }
    }
else:
    container_export_like = {
        "containerVersion": {
            "tag": tags,
            "trigger": triggers,
            "variable": variables,
            "builtInVariable": builtins,
        }
    }


In [ ]:
import re, csv, yaml
from collections import Counter, defaultdict

# ---- Policy helpers ----
CONSENT_EXEMPT_NAME_PREFIX = "Utility - "
GA4_TYPES = {"gaawc", "gaawe"}  # GA4 Config / GA4 Event
GA4_EXEMPT_NAME_PREFIX = "GA4 - "   # GA4 tags by naming convention are exempt


VAR_REF_RE = re.compile(r"\{\{([^}]+)\}\}")

def _collect_var_refs_from_obj(obj, acc: set):
    """Recursively walk GTM resource structure and collect {{var}} references."""
    if isinstance(obj, str):
        for m in VAR_REF_RE.findall(obj):
            acc.add(m)
    elif isinstance(obj, dict):
        for v in obj.values():
            _collect_var_refs_from_obj(v, acc)
    elif isinstance(obj, (list, tuple)):
        for v in obj:
            _collect_var_refs_from_obj(v, acc)

def var_refs_from_tag(tag) -> set[str]:
    """
    Return all variable names referenced anywhere in the tag:
    - parameters (including nested list/map)
    - fields to set, etc.
    """
    acc: set[str] = set()
    # tags usually store templates in "parameter", "monitoringMetadata", etc.; safest is to scan whole tag
    _collect_var_refs_from_obj(tag.get("parameter") or [], acc)
    _collect_var_refs_from_obj(tag.get("monitoringMetadata") or {}, acc)
    # if you want to be super safe, uncomment the next line to walk the whole tag:
    # _collect_var_refs_from_obj(tag, acc)
    return acc

def var_refs_from_variable(v) -> set[str]:
    """Return all variable references inside a variable definition."""
    acc: set[str] = set()
    _collect_var_refs_from_obj(v.get("parameter") or [], acc)
    return acc

def var_refs_from_trigger(tr) -> set[str]:
    """
    Return all variable references used in a trigger:
    - main parameters
    - filter / condition parameters (where GTM stores arg0/arg1/etc.)
    """
    acc: set[str] = set()
    _collect_var_refs_from_obj(tr.get("parameter") or [], acc)
    _collect_var_refs_from_obj(tr.get("filter") or [], acc)
    _collect_var_refs_from_obj(tr.get("condition") or [], acc)
    return acc


# tags from external vendors – we don't enforce our own ga4_/utility_/snowplow_ prefixes on these
VENDOR_NAME_PREFIXES = [
    "Adtraction",
    "Bonusway",
    "Custobar",
    "Kinesso",
    "Meta",          # "Meta - Pixel", etc.
    "Facebook",
    "Google Ads",
    "LinkedIn",
    "TikTok",
    "Criteo",
    "Pinterest",
    "Microsoft Ads",
    "Microsoft",
]


def is_vendor_tag_name(name: str) -> bool:
    n = (name or "").strip()
    return any(n.startswith(prefix) for prefix in VENDOR_NAME_PREFIXES)


def tag_type(tag):
    return (tag.get("type") or "").lower()

def consent_info(tag):
    """
    Read consentSettings from a tag and normalize:
    - status: UPPERCASE string (e.g. 'NEEDED', 'NOT_SET')
    - types: lowercase set of consent types (e.g. {'ad_storage','analytics_storage'})
    """
    cs = tag.get("consentSettings") or {}

    raw_status = cs.get("consentStatus") or "NOT_SET"
    status = str(raw_status).upper()

    raw_types = cs.get("consentType") or cs.get("consentTypes") or []
    types = set()

    # Case 1: simple string
    if isinstance(raw_types, str):
        types.add(raw_types.lower())

    # Case 2: list of strings or dicts
    elif isinstance(raw_types, (list, tuple, set)):
        for item in raw_types:
            if isinstance(item, str):
                types.add(item.lower())
            elif isinstance(item, dict):
                # common pattern: {"consentType": "ad_storage"} or {"type": "...", "value": "..."}
                ct = item.get("consentType") or item.get("value") or item.get("type")
                if isinstance(ct, str):
                    types.add(ct.lower())

    # Case 3: dict (your Google Ads example: {"list": [...], "type": "list"})
    elif isinstance(raw_types, dict):
        # If it has a "list" key with inner items, parse those
        inner_list = raw_types.get("list")
        if isinstance(inner_list, list):
            for item in inner_list:
                if isinstance(item, str):
                    types.add(item.lower())
                elif isinstance(item, dict):
                    ct = item.get("consentType") or item.get("value") or item.get("type")
                    if isinstance(ct, str):
                        types.add(ct.lower())
        else:
            # Fallback: try values/keys
            for k, v in raw_types.items():
                if isinstance(v, str):
                    types.add(v.lower())
                elif isinstance(k, str):
                    types.add(k.lower())

    return status, types



# our rule: Utility-* exempt, GA4 needs analytics_storage, everything else needs both.
def required_consent_for_tag(tag) -> set[str]:
    """
    Consent rules:
    - 'Utility - *' → exempt
    - 'GA4 - *'    → exempt (per your request)
    - GA4 types    → analytics_storage
    - Everything else → ad_storage + analytics_storage
    """
    name = (tag.get("name") or "")

    # 1) Utility tags: fully exempt
    if name.startswith(CONSENT_EXEMPT_NAME_PREFIX):
        return set()

    # 2) GA4 tags by naming convention: exempt (you said GA4 - … should be exempt)
    if name.startswith(GA4_EXEMPT_NAME_PREFIX):
        return set()

    # 3) Other GA4 tags by type: need analytics_storage only
    if tag_type(tag) in GA4_TYPES:
        return {"analytics_storage"}

    # 4) Everything else: ad_storage + analytics_storage
    return {"ad_storage", "analytics_storage"}
    

def tag_html(tag):
    params = {p.get("key"): p.get("value") for p in (tag.get("parameter") or []) if "key" in p}
    return params.get("html", "") or ""

def list_refs_from_tag(tag):
    firing = set(tag.get("firingTriggerId", []) or [])
    blocking = set(tag.get("blockingTriggerId", []) or [])
    # variable refs will now come from var_refs_from_tag()
    vrefs = var_refs_from_tag(tag)
    return firing, blocking, vrefs


def infer_prefixes(names, k=3):
    tokens = []
    for n in names:
        if not n: continue
        m = re.match(r"^([A-Za-z]+?)[\s:\-_/]", n)
        tokens.append(m.group(1).lower() + "_" if m else (n[:3].lower() + "_"))
    counts = Counter(tokens)
    return [p for p, _ in counts.most_common(k) if p]

# ---- The audit itself ----
def audit(container_export_like: dict, rules: dict):
    cv = container_export_like.get("containerVersion", {})
    tags = cv.get("tag", []) or []
    triggers = cv.get("trigger", []) or []
    variables = cv.get("variable", []) or []
    builtins = cv.get("builtInVariable", []) or []

    findings = []
    sev = rules["severity"]
    allowed_name_re = re.compile(rules["naming"]["allowed_pattern"])

    # naming prefixes
    req_pref = rules["naming"]["required_prefixes"].copy()
    # only auto-infer for types you have not set explicitly
    if not req_pref.get("tag"):
        req_pref["tag"] = infer_prefixes([t.get("name","") for t in tags])
    if not req_pref.get("trigger"):
        req_pref["trigger"] = infer_prefixes([tr.get("name","") for tr in triggers])
    if not req_pref.get("variable"):
        req_pref["variable"] = infer_prefixes([v.get("name","") for v in variables])

    # duplicate names
    if rules["naming"]["disallow_duplicates"]:
        for kind, names in [
            ("tag", [t.get("name","") for t in tags]),
            ("trigger", [tr.get("name","") for tr in triggers]),
            ("variable", [v.get("name","") for v in variables]),
        ]:
            for n, c in Counter(names).items():
                if n and c > 1:
                    findings.append({
                        "severity": sev["naming"],
                        "rule": "duplicate_name",
                        "kind": kind,
                        "name": n,
                        "detail": "Duplicate name",
                    })

    # name pattern + prefix
    def check_name(kind, name):
        if not name:
            findings.append({
                "severity": sev["naming"],
                "rule": "missing_name",
                "kind": kind,
                "name": "",
                "detail": "Missing name",
            })
            return
        if not allowed_name_re.match(name):
            findings.append({
                "severity": sev["naming"],
                "rule": "name_pattern",
                "kind": kind,
                "name": name,
                "detail": "Violates allowed_pattern",
            })
        prefixes = req_pref.get(kind, [])
        if prefixes and not any(name.startswith(p) for p in prefixes):
            findings.append({
                "severity": sev["naming"],
                "rule": "prefix_required",
                "kind": kind,
                "name": name,
                "detail": f"Should start with one of: {prefixes}",
            })

    # tags: skip vendor tags for naming prefixes
    for t in tags:
        name = t.get("name","")
        if is_vendor_tag_name(name):
            continue
        check_name("tag", name)

    for tr in triggers:
        check_name("trigger", tr.get("name",""))
    for v in variables:
        check_name("variable", v.get("name",""))

    # consent init/update triggers must exist
    if rules["consent"]["require_consent_init_update_triggers"]:
        kinds = {tr.get("type") for tr in triggers}
        if "consentInitialization" not in kinds:
            findings.append({
                "severity": sev["consent"],
                "rule": "consent_init_missing",
                "kind": "trigger",
                "name": "",
                "detail": "Missing Consent Initialization trigger",
            })
        if "consentUpdate" not in kinds:
            findings.append({
                "severity": sev["consent"],
                "rule": "consent_update_missing",
                "kind": "trigger",
                "name": "",
                "detail": "Missing Consent Update trigger",
            })

    # optional: consent default in HTML – OFF for you (CMP handles this)
    if rules["consent"].get("check_consent_default_presence", False):
        cdef_re = re.compile(rules["consent"]["consent_default_regex"])
        if not any(cdef_re.search(tag_html(t)) for t in tags if tag_type(t) == "html"):
            findings.append({
                "severity": sev["consent"],
                "rule": "consent_default_missing",
                "kind": "tag",
                "name": "",
                "detail": "No gtag('consent','default',...) seen in GTM",
            })

    # per-tag consent enforcement
    if rules["consent"]["require_tag_consent_settings"]:
        for t in tags:
            need = required_consent_for_tag(t)
            if not need:
                continue  # Utility / GA4-exempt
            status, ctypes = consent_info(t)
    
            # treat NEEDED / REQUIRED as acceptable
            if status not in {"NEEDED", "REQUIRED"}:
                findings.append({
                    "severity": sev["consent"],
                    "rule": "tag_consent_status",
                    "kind": "tag",
                    "name": t.get("name",""),
                    "detail": f"Must set consentStatus=NEEDED/REQUIRED for {sorted(need)} (got {status})",
                })
    
            missing = need - ctypes
            if missing:
                findings.append({
                    "severity": sev["consent"],
                    "rule": "tag_consent_types",
                    "kind": "tag",
                    "name": t.get("name",""),
                    "detail": f"Missing consent types: {sorted(missing)}",
                })


    # hygiene: unused triggers/variables (deep scan using recursive extractor)
    if rules["hygiene"]["flag_unused_triggers"] or rules["hygiene"]["flag_unused_variables"]:
        used_triggers: set[str] = set()
        used_variables: set[str] = set()
    
        # 1) Tags → triggers + variables (anywhere inside tag config)
        for t in tags:
            f, b, vrefs = list_refs_from_tag(t)
            used_triggers |= f | b
            used_variables |= vrefs
    
        # 2) Variables → variables (variables used inside other variables)
        for v in variables:
            used_variables |= var_refs_from_variable(v)
    
        # 3) Triggers → variables (variables in trigger conditions)
        for tr in triggers:
            used_variables |= var_refs_from_trigger(tr)
    
        if rules["hygiene"]["flag_unused_triggers"]:
            for tr in triggers:
                tid = tr.get("triggerId")
                nm = tr.get("name", "")
                if tid and tid not in used_triggers:
                    findings.append({
                        "severity": sev["hygiene"],
                        "rule": "unused_trigger",
                        "kind": "trigger",
                        "name": nm,
                        "detail": f"TriggerId {tid} not referenced by any tag",
                    })
    
        if rules["hygiene"]["flag_unused_variables"]:
            for v in variables:
                nm = v.get("name", "")
                if nm and (nm not in used_variables):
                    findings.append({
                        "severity": sev["hygiene"],
                        "rule": "unused_variable",
                        "kind": "variable",
                        "name": nm,
                        "detail": "Variable not referenced by any tag, trigger, or other variable",
                    })


    summary = {
        "counts": {
            "tags": len(tags),
            "triggers": len(triggers),
            "variables": len(variables),
            "findings": len(findings),
        },
        "prefix_suggestions": req_pref,
    }

    return findings, summary


# ---------- write_report() ----------
def write_report(findings, summary, out_yml="gtm_audit.report.yml", out_csv="gtm_audit_findings.csv"):
    report = {
        "meta": {"tool": "gtm_audit", "version": "1.2"},
        "summary": summary,
        "findings": findings,
    }
    with open(out_yml, "w", encoding="utf-8") as f:
        yaml.safe_dump(report, f, sort_keys=False, allow_unicode=True)

    cols = ["severity","rule","kind","name","detail"]
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        for row in findings:
            w.writerow({c: row.get(c, "") for c in cols})


In [ ]:
# ---------- Prefix inference (only if missing in RULES) ----------
def infer_prefixes(names, k=3):
    tokens = []
    for n in names:
        if not n:
            continue
        m = re.match(r"^([A-Za-z]+?)[\s:\-_/]", n)
        tokens.append(m.group(1) + " - " if m else (n.split(" ")[0] + " - "))
    counts = Counter(tokens)
    return [p for p, _ in counts.most_common(k) if p]

RULES = {
  "naming": {
    "required_prefixes": {
        "tag": ["GA4 - ", "Utility - ", "Snowplow - "],   # our tag prefixes
        "trigger": ["Click - ", "Event - ", "Link - ", "All pages - ", "Blocking - ", "Link Click - ", "Form - ", "Cookie - ", "URL - ", "Lookup - ", "Visibility - "],   # our trigger prefixes
        "variable": ["User - ","CJS - ", "DL - ", "CST - ", "RDM - ", "Click Element - ", "Lookup - ", "URL - ", "Cookie - ", "Boolean - ", "CT - ", "JS - "],        # our variables prefixes
    #CT for Custom Template variable
    #CJS for Custom JavaScript variable
    #DL for DataLayer variable
    #CST for Constant variable
    #JS for JavaScript variable
    #Click Element for Click Element variable
    #Lookup for Lookup variable
    #URL for URL variable
        
    },
    # allow letters, digits, spaces, punctuation, and €
    "allowed_pattern": r"^[\w\s\.\-:/\(\)\{\}\[\]\|,+&%€£$¥]+$",
    "disallow_duplicates": True,
  },
  "consent": {
    "require_tag_consent_settings": True,
    "require_consent_init_update_triggers": True,
    "check_consent_default_presence": False,
    "consent_default_regex": r'gtag\(\s*[\'"]consent[\'"]\s*,\s*[\'"]default[\'"]',
  },
  "hygiene": {
    "flag_unused_triggers": True,
    "flag_unused_variables": True,
    "html_requires_comment_marker": False,
    "html_marker_regex": r"data-consent|type=['\"]text/plain['\"]",
  },
  "severity": {"naming":"warn","consent":"error","hygiene":"warn"},
}

print(RULES["naming"]["required_prefixes"])

In [ ]:
container_export_like = {
    "containerVersion": {
        "tag": tags,
        "trigger": triggers,
        "variable": variables,
        "builtInVariable": builtins,
    }
}

findings, summary = audit(container_export_like, RULES)
write_report(findings, summary, out_yml=out_yml, out_csv=out_csv)

print("✅ Audit complete:", summary["counts"])
print("📄 CSV:", out_csv)
print("📄 YAML:", out_yml)


print(summary["prefix_suggestions"])

from collections import Counter
print("Top rules:", Counter(f["rule"] for f in findings).most_common(10))
for f in findings[:10]:
    print(f"- [{f['severity']}] {f['rule']} | {f['kind']}='{f['name']}' → {f['detail']}")
